In [1]:
import os
from collections import defaultdict

In [2]:
refseq_to_chm_number = {}

with open("../../chm_to_acc_numbers.tsv") as fh:
    for line in fh:
        if line.startswith('#'):
            continue
        line = line.strip().split("\t")
        refseq_to_chm_number[line[6]] = line[0]

In [6]:
#transcript

## dict with search results

# {read_n:{chr_n: {trans_n: str, chr_n: str, coordinates : [],total_length : int}}}

reads_summary = defaultdict()

## m8 == outftmt6 
## query_seq_id => target_seq_id => chm_number => identity => length => mismatch => gaps => 
## ==> start_query => stop_query => start_target => stop_target => e_value => bit-score
with open("../../human_transcript_pacbio_reads/result_long_reads_human_t2t_match.m8") as fh:
    for line in fh:
        line = line.strip().split("\t")
        read = line[0]
        identity = float(line[2])
        chr_n = refseq_to_chm_number[line[1]]
        transcript = line[0].split('_')[4]
        start = int(line[-4])
        stop = int(line[-3])
        start_read = int(line[-6])
        stop_read = int(line[-5])
        if start > stop:
#             print('Stop < start, changing...')
#             print(start, stop)
            start, stop = stop, start
#             print('Change to: ', start, stop)
        if start_read > stop_read:
            start_read, stop_read = stop_read, start_read
        if ((stop - start) < 200) or (identity < 0.8):
            continue
        if not read in reads_summary.keys():
            reads_summary[read] = {}
        if not chr_n in reads_summary[read].keys():
#             print('Adding initial coords to chr', chr_n, start, stop)
            reads_summary[read][chr_n] = {
                'transcript_n': transcript,
                'chr_n': chr_n,
                'coordinates_genome': [f"{start}:{stop}"],
                'coordinates_read': [f"{start_read}:{stop_read}"],
                'total_length_genome': stop - start,
                'total_length_read' : stop_read - start_read
            }
        else:
#             nested_longer = None
#             nested_longer_index = None
            not_nested = None
            #difference = None
            for i, coord in enumerate(reads_summary[read][chr_n]['coordinates_genome']):
                start_in = int(coord.split(':')[0])
                stop_in = int(coord.split(':')[1])
#                 print(f'Coords check: {start_in}:{stop_in} to {start}:{stop}')
#                 if start < start_in and stop > stop_in:
#                     difference = (stop - start) - (stop_in - start_in)
#                     nested_longer = True
#                     nested_longer_index = i
#                 elif start < start_in and start_in < stop < stop_in:
#                     if (stop - start) > (stop_in - start_in):
#                         difference = (stop - start) - (stop_in - start_in)
#                         nested_longer = True
#                         nested_longer_index = i
#                 elif start_in < start < stop and stop > stop_in:
#                     if (stop - start) > (stop_in - start_in):
#                         difference = (stop - start) - (stop_in - start_in)
#                         nested_longer = True
#                         nested_longer_index = i
                if (start < start_in) and (stop < start_in):
#                     print('Not Nested 1! Continue check...')
                    not_nested = True
                elif (start > stop_in) and (stop > stop_in):
                    not_nested = True
#                     print('Not nested 2! Continue check...')
                else:
#                     print('Nested! Breaking...')
                    not_nested = False
                    break
#             if nested_longer:
#                 reads_summary[read][chr_n]['coordinates'][nested_longer_index] = f'{start}:{stop}'
#                 reads_summary[read][chr_n]['total_length'] += difference
            if not_nested:
                reads_summary[read][chr_n]['coordinates_genome'].append(f'{start}:{stop}')
                reads_summary[read][chr_n]['total_length_genome'] += stop - start
                if f'{start_read}:{stop_read}' in reads_summary[read][chr_n]['coordinates_read']:
                    continue
                else:
                    not_nested_read = None
                    for i, coord in enumerate(reads_summary[read][chr_n]['coordinates_read']):
                        start_in = int(coord.split(':')[0])
                        stop_in = int(coord.split(':')[1])
                        if (start_read < start_in) and (stop_read < start_in):
                            not_nested_read = True
                        elif (start_read > stop_in) and (stop_read > stop_in):
                            not_nested_read = True
                        else:
                            not_nested_read = False
                            break
                    if not_nested_read:
                        reads_summary[read][chr_n]['coordinates_read'].append(f'{start_read}:{stop_read}')
                        reads_summary[read][chr_n]['total_length_read'] += stop_read - start_read

In [7]:
import json
with open('../data/reads_match_coords.json', 'w', encoding='utf-8') as f:
    json.dump(reads_summary, f, ensure_ascii=False, indent=2)

In [8]:
reads_summary_without_coords = {}

for read, chr_info in reads_summary.items():
    print(read)
    for chr_n, info in chr_info.items():
        read_key = f"{read}_chr_{chr_n}"
        print(read_key)
        reads_summary_without_coords[read_key] = [chr_n, info['transcript_n'], str(info['total_length_genome']), str(info['total_length_read'])]

SRR15347854.1597595_len_2595_transcript_1_ident_0.466_length_171_start_183_stop_34
SRR15347854.1597595_len_2595_transcript_1_ident_0.466_length_171_start_183_stop_34_chr_22
SRR15347854.1597595_len_2595_transcript_1_ident_0.466_length_171_start_183_stop_34_chr_7
SRR15347854.1597595_len_2595_transcript_1_ident_0.466_length_171_start_183_stop_34_chr_17
SRR15347854.613575_len_3587_transcript_4_ident_0.938_length_129_start_617_stop_745
SRR15347854.613575_len_3587_transcript_4_ident_0.938_length_129_start_617_stop_745_chr_1
SRR15347854.613575_len_3587_transcript_4_ident_0.938_length_129_start_617_stop_745_chr_10
SRR15347854.291150_len_5659_transcript_1_ident_0.466_length_171_start_178_stop_29
SRR15347854.291150_len_5659_transcript_1_ident_0.466_length_171_start_178_stop_29_chr_22
SRR15347854.291150_len_5659_transcript_1_ident_0.466_length_171_start_178_stop_29_chr_17
SRR15347854.746699_len_3364_transcript_4_ident_0.926_length_129_start_1280_stop_1408
SRR15347854.746699_len_3364_transcript_4_

In [9]:
with open('../data/reads_match_summary.tsv', 'w') as fw:
    fw.write('#read\tchromosome\ttranscript\ttotal_length_genome\ttotal_length_read\n')
    for read, summary in reads_summary_without_coords.items():
        line_to_write = "\t".join(summary)
        fw.write(f"{read}\t{line_to_write}\n")

In [95]:
## extract reads with transcript 4.1 to search with blat

def fasta_reader_yield(path_to_fasta_file):
    header = None
    with open(path_to_fasta_file) as fh:
        for i, line in enumerate(fh):
            line = line.strip()
            if line.startswith(">"):
                if header:
                    yield header,"".join(seq)
                header = line[1:]
                seq = []
            else:
                seq.append(line)
        if header:
            yield header,"".join(seq)
            
reads_headers = []

with open("../mmseq_search_results/transcript_4_1_to_human_transc_reads.m8") as fh:
    for line in fh:
        line = line.strip().split("\t")
        if line[1] not in reads_headers and float(line[2]) > 0.9 and int(line[3]) > 200:
            print(line)
            reads_headers.append(line[1])

['ATrav-08-04-013-HS', 'SRR15347854.431657', '1.000', '375', '0', '0', '715', '341', '3236', '3610', '2.427E-80', '270']
['ATrav-08-04-013-HS', 'SRR15347854.1148349', '1.000', '375', '0', '0', '715', '341', '2348', '2722', '2.427E-80', '270']
['ATrav-08-04-013-HS', 'SRR15347854.1220344', '1.000', '375', '0', '0', '715', '341', '149', '523', '8.573E-80', '268']
['ATrav-08-04-013-HS', 'SRR15347854.1652764', '1.000', '375', '0', '0', '715', '341', '3472', '3846', '2.209E-79', '267']
['ATrav-08-04-013-HS', 'SRR15347854.1294088', '0.999', '375', '0', '0', '715', '341', '345', '719', '4.152E-79', '266']
['ATrav-08-04-013-HS', 'SRR15347854.293923', '0.995', '375', '2', '0', '715', '341', '1049', '1423', '1.070E-78', '265']
['ATrav-08-04-013-HS', 'SRR15347854.41151', '0.990', '375', '4', '0', '715', '341', '3869', '3495', '3.780E-78', '263']
['ATrav-08-04-013-HS', 'SRR15347854.774109', '0.987', '375', '5', '0', '715', '341', '3412', '3038', '7.104E-78', '263']
['ATrav-08-04-013-HS', 'SRR153478

In [91]:
reads_headers

['SRR15347854.431657',
 'SRR15347854.1148349',
 'SRR15347854.1220344',
 'SRR15347854.1652764',
 'SRR15347854.1294088',
 'SRR15347854.293923',
 'SRR15347854.41151',
 'SRR15347854.774109',
 'SRR15347854.740620',
 'SRR15347854.1120963',
 'SRR15347854.746699',
 'SRR15347854.1633414',
 'SRR15347854.1698311',
 'SRR15347854.613575',
 'SRR15347854.1173950',
 'SRR15347854.10804',
 'SRR15347854.1212119',
 'SRR15347854.85586']

In [97]:
fasta_reads = '../../human_transcript_pacbio_reads/SRR15347854.fasta'
fasta_found = "../../transript_4_1_pacbio_reafs.fasta"

reads = {}

with open(fasta_found, 'a') as fa:
    for h, s in fasta_reader_yield(fasta_reads):
        head = h.split()[0]
        for header in reads_headers:
            if head == header:
                line_to_write = f">{h}\n{s}\n"
                fa.write(line_to_write)